# BAA MAEs
### Section 2.1 of the Supplementary Information (Table 3)

# Qiskit config

In [ ]:
from qiskit import IBMQ, execute, transpile
from qiskit.providers.aer.backends import AerSimulator

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research') # 'ibm-q'

In [ ]:
backend_1  = provider.get_backend('ibmq_qasm_simulator')
#backend_2  = provider.get_backend('ibmq_casablanca') casablanca is no more. Farewell, old friend.
backend_3  = provider.get_backend('ibmq_jakarta')
backend_4  = provider.get_backend('ibm_perth')

shots      = 8192

"""
    Select the backends that will be compared.
"""

backends = [ backend_1 ]

# Experiment procedures

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from qclib.state_preparation.baa_schmidt import initialize

def counts(transpiled_circuit):
    count_ops = transpiled_circuit.count_ops()
    n_cx = 0
    if 'cx' in count_ops:
        n_cx = count_ops['cx']
    n_dp = transpiled_circuit.depth()

    return n_cx, n_dp

def fidelity_and_counts(state, l):
    circuit = initialize(state, max_fidelity_loss=l, strategy='brute_force', use_low_rank=True)
    transpiled_circuit = transpile(circuit, basis_gates=['u1','u2','u3', 'cx'], optimization_level=3)
    backend = AerSimulator()
    transpiled_circuit.save_statevector()
    ket = backend.run(transpiled_circuit).result().get_statevector()
    bra = np.conj(state)

    n_cx, n_dp = counts(transpiled_circuit)

    return np.abs(bra.dot(ket))**2, n_cx, n_dp, ket

def measurement(circuit):
    n = len(circuit.qubits)
    circuit.measure_all()

    job = execute(circuit, backend, shots=shots, optimization_level=3)
    
    counts = job.result().get_counts(circuit)
    v = sum(counts.values())
    
    counts2 = {}
    for m in range(2**n):
        pattern = '{:0{}b}'.format(m, n)
        if pattern in counts:
            counts2[pattern] = counts[pattern]
        else:
            counts2[pattern] = 0.0
            
    return { key : value/v for (key, value) in counts2.items() }
    
def run_circuit(state, l=0.0):
    circuit = initialize(state, max_fidelity_loss=l, strategy='brute_force', use_low_rank=False)
    
    prob = measurement(circuit)
    
    return np.array([val for key, val in prob.items()])

# Experiment

In [ ]:
reps = 10
n_list = [7]

In [ ]:
states = {}

# Random complex input vector.
for n in n_list:
    rnd = np.random.RandomState(42)
    state = rnd.rand(2**n) + rnd.rand(2**n) * 1j
    states[2**n] = state/np.linalg.norm(state)

ideals = {}
result = {}
fidelity = {}
for n in n_list:
    print('\nn =', n)
    
    input_state = states[2**n]
    
    ideals[n] = np.power(np.abs(input_state), 2)

    result[n] = {}
    fidelity[n] = {}
    for j, backend in enumerate(backends):
        backend_name = backend.name()
        backend_config = backend.configuration()
        backend_qubits = backend_config.n_qubits

        print('\nExperiments using {0} backend, with {1} qubits available.'.format(backend_name, backend_qubits))

        result[n][backend_name] = {}
        fidelity[n][backend_name] = {}
        loss = [0.0, 0.12, 0.18, 0.19, 0.22, 0.23, 0.24, 0.25, 0.26]
        for l in loss:
            print('max. l =', l, 'run: ', end='')

            probs = []
            for i in range(reps):
                print(str(i)+' ', end='')
                probs.append( run_circuit(input_state, l) )

            fy, n_cx, n_dp, ket = fidelity_and_counts(input_state, l)
            print('fidelity:', round(fy,5), '\tcnots:', n_cx, '\tdepth:', n_dp)
            result[n][backend_name][l] = probs
            fidelity[n][backend_name][l] = fy



# Print Results

In [ ]:
def print_result(ideal, probs):
    n = int(np.log2(len(ideal)))
        
    for r, prob in probs.items():
        for i, p in enumerate(prob):
            mae = np.sum( np.abs( p - ideal ) ) / 2**n
            print('max. l =', r, '\tMAE',i,'=',mae)
    print('')
    for r, prob in probs.items():
        maes = []
        for i, p in enumerate(prob):
            mae = np.sum( np.abs( p - ideal ) ) / 2**n
            maes.append( mae )
        avg = np.mean(maes)
        std = np.std(maes)
        print('max. l =', r, '\tAVG.MAE =', round(avg,7), '\tSTD.MAE =', std)

for n in n_list:
    print('\nn =', n)
    for j, backend in enumerate(backends):
        print('\nbackend =', backend.name(), '\n')
        print_result(ideals[n], result[n][backend.name()])
